In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_ds = train_datagen.flow_from_directory(
    "dataset_new/train",
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical',
    shuffle=True
)

val_ds = val_datagen.flow_from_directory(
    'dataset_new/val',
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)


Found 28709 images belonging to 7 classes.
Found 6217 images belonging to 7 classes.


In [3]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.3),  # Increased rotation
  layers.RandomZoom(0.3),     # Increased zoom
  layers.RandomTranslation(0.2, 0.2),
  layers.RandomShear(0.2),    # Add shear transformation
  # layers.RandomErasing(0.5)  # Add random erasing (requires tf.keras.layers.experimental)
 ])

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Dropout,
                                     Flatten, Dense, BatchNormalization)

model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(48, 48, 1), padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Conv2D(256, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 classes for FER2013
])

/Users/amalkrishnamu/Documents/PROJECTS/Emotion_Detection/emo_env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [6]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     4,719,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,094,151 (19.43 MB)

 Trainable params: 5,093,255 (19.43 MB)

 Non-trainable params: 896 (3.50 KB)

In [7]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [8]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=early_stop
)

Epoch 1/50


/Users/amalkrishnamu/Documents/PROJECTS/Emotion_Detection/emo_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - accuracy: 0.2221 - loss: 3.0370  

/Users/amalkrishnamu/Documents/PROJECTS/Emotion_Detection/emo_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


449/449 ━━━━━━━━━━━━━━━━━━━━ 130s 287ms/step - accuracy: 0.2222 - loss: 3.0349 - val_accuracy: 0.1610 - val_loss: 1.9775
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 131s 291ms/step - accuracy: 0.2783 - loss: 1.7636 - val_accuracy: 0.2932 - val_loss: 1.7177
Epoch 3/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 128s 284ms/step - accuracy: 0.3232 - loss: 1.6882 - val_accuracy: 0.3603 - val_loss: 1.6326
Epoch 4/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 124s 276ms/step - accuracy: 0.3547 - loss: 1.6259 - val_accuracy: 0.2779 - val_loss: 1.8062
Epoch 5/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 123s 273ms/step - accuracy: 0.3632 - loss: 1.5879 - val_accuracy: 0.4005 - val_loss: 1.5486
Epoch 6/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 125s 279ms/step - accuracy: 0.3839 - loss: 1.5557 - val_accuracy: 0.4147 - val_loss: 1.5411
Epoch 7/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 133s 296ms/step - accuracy: 0.3911 - loss: 1.5388 - val_accuracy: 0.3182 - val_loss: 1.7111
Epoch 8/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 126s 281ms/step - accuracy: 0.3940 - loss: 1.53

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Only rescaling — no augmentation for test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_ds = test_datagen.flow_from_directory(
    "dataset_new/test",         # ← Change this to your actual test folder path
    target_size=(48, 48),
    color_mode='grayscale', # Match training mode
    batch_size=64,
    class_mode='categorical',
    shuffle=False           # Don't shuffle — useful for accurate evaluation
)


Found 7178 images belonging to 7 classes.


In [20]:
loss, accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

113/113 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.4997 - loss: 1.2448
Test Accuracy: 57.37%


In [22]:
model.save("model5.h5")